---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 4 - Document Similarity & Topic Modelling

## Part 1 - Document Similarity

For the first part of this assignment, you will complete the functions `doc_to_synsets` and `similarity_score` which will be used by `document_path_similarity` to find the path similarity between two documents.

The following functions are provided:
* **`convert_tag:`** converts the tag given by `nltk.pos_tag` to a tag used by `wordnet.synsets`. You will need to use this function in `doc_to_synsets`.
* **`document_path_similarity:`** computes the symmetrical path similarity between two documents by finding the synsets in each document using `doc_to_synsets`, then computing similarities using `similarity_score`.

You will need to finish writing the following functions:
* **`doc_to_synsets:`** returns a list of synsets in document. This function should first tokenize and part of speech tag the document using `nltk.word_tokenize` and `nltk.pos_tag`. Then it should find each tokens corresponding synset using `wn.synsets(token, wordnet_tag)`. The first synset match should be used. If there is no match, that token is skipped.
* **`similarity_score:`** returns the normalized similarity score of a list of synsets (s1) onto a second list of synsets (s2). For each synset in s1, find the synset in s2 with the largest similarity value. Sum all of the largest similarity values together and normalize this value by dividing it by the number of largest similarity values found. Be careful with data types, which should be floats. Missing values should be ignored.

Once `doc_to_synsets` and `similarity_score` have been completed, submit to the autograder which will run `test_document_path_similarity` to test that these functions are running correctly. 

*Do not modify the functions `convert_tag`, `document_path_similarity`, and `test_document_path_similarity`.*

In [1]:
# import nltk 
# nltk.download()

In [180]:
# testing synsets and pos tokens
doc = "THE DOG IS BARKING the"
doc1 = doc.lower()
token = nltk.word_tokenize(doc1)
pos = nltk.pos_tag(token)
pos_ = [w[1] for w in pos]
pos_tag = [convert_tag(p) for p in pos_]
print('tokens:', token, 'pos_tags: ', pos_tag)
wn.synsets(token[0], pos_tag[0])
a = doc_to_synsets(doc)
a

tokens: ['the', 'dog', 'is', 'barking', 'the'] pos_tags:  [None, 'n', 'v', 'v', None]


[Synset('dog.n.01'), Synset('be.v.01'), Synset('bark.v.01')]

In [12]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    from itertools import zip_longest
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    
    doc = doc.lower() # Lower case the document
    token = nltk.word_tokenize(doc) # Tokenize
    pos = nltk.pos_tag(token) # generate POS tags for tokens
    pos_ = [w[1] for w in pos] # transform into a list 
    pos_tag = [convert_tag(p) for p in pos_] # use dictionary to convert to POS tags format accepted by wordnet
    
    syn = []
    for tk, po in zip_longest(token, pos_tag): # zip longest zips both and iterates over the longest length
        if wn.synsets(str(tk), pos=po): # form synsets for each token and corresponding POS tag
            syn.append(wn.synsets(str(tk), pos=po)[0])
        
    
    return syn


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    sim=0
    similarity = []
    for si in s1:
        dis = 0
        for sii in s2:
            if si.path_similarity(sii): # if there is path similarity then assign to variable 
                sim = si.path_similarity(sii)
            if dis < sim: # dis keeps the largest similarity distance and appends to similarity list for given si
                dis = sim
        similarity.append(dis)
    
    
        
    return (sum(similarity)/len(similarity))


def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

### test_document_path_similarity

Use this function to check if doc_to_synsets and similarity_score are correct.

*This function should return the similarity score as a float.*

In [8]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

<br>
___
`paraphrases` is a DataFrame which contains the following columns: `Quality`, `D1`, and `D2`.

`Quality` is an indicator variable which indicates if the two documents `D1` and `D2` are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [9]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


In [ ]:
# Testing different logics
from itertools import zip_longest
a = paraphrases.iloc[0]['D1']#.split(' ')
b = paraphrases.iloc[0]['D2']#.split(' ')

s1 = doc_to_synsets(a)
s2 = doc_to_synsets(b)

# for ax, bx in zip_longest(aa,bb):
#     print (ax,bx)

for si in s2:
    if si.name().split('.')[0] == 'executive':
        print(si.name().split('.')[0])
# sim=0
# similarity = []
# for si in s1:
#     dis = 0
#     for sii in s2:
#         if si.path_similarity(sii):# and si != 'None' and sii != 'None':
#             sim = si.path_similarity(sii)
#         if dis < sim:
#             dis = sim
#     similarity.append(dis)
    


___

### most_similar_docs

Using `document_path_similarity`, find the pair of documents in paraphrases which has the maximum similarity score.

*This function should return a tuple `(D1, D2, similarity_score)`*

In [ ]:
def most_similar_docs():

    result = []
    for index, row in paraphrases.iterrows():
        result.append(document_path_similarity(row['D1'], row['D2']))
    paraphrases['similarity_score'] = result
    (D1, D2, similarity_score) = paraphrases[paraphrases['similarity_score'] == paraphrases['similarity_score'].max()].loc[13, 'D1':'similarity_score']
    return (D1, D2, similarity_score)

### label_accuracy

Provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. Let the classifier rule be that if the score is greater than 0.75, label is paraphrase (1), else label is not paraphrase (0). Report accuracy of the classifier using scikit-learn's accuracy_score.

*This function should return a float.*

In [87]:
def label_accuracy():
    from sklearn.metrics import accuracy_score
    
    paraphrases['label'] = paraphrases['similarity_score'].apply(lambda x: 1 if x>0.75 else 0)
        
    
    return (accuracy_score(paraphrases['Quality'], paraphrases['label'])) 

In [96]:
paraphrases.sort_values(by = 'similarity_score', ascending =False)

,Quality,D1,D2,similarity_score,label
13,1,"""Indeed, Iran should be put on notice that eff...","""Iran should be on notice that attempts to rem...",0.950292,1
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...,0.929902,1
15,0,US Special Forces troops are training Colombia...,U.S. Special Forces troops are training Colomb...,0.872292,1
1,1,After more than two years' detention under the...,After more than two years in detention by the ...,0.855833,1
5,1,Their difference was over whether the court sh...,Their difference was over whether the court sh...,0.820710,1
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec...",0.818210,1
6,1,The only announced Republican to replace Davis...,So far the only declared major party candidate...,0.767063,1
7,1,"Druce will face murder charges, Conte said.",Conte said Druce will be charged with murder.\n,0.761667,1
19,1,The NASD also alleges Young flew multiple time...,The NASD alleges that the analyst flew multipl...,0.721776,0
9,1,Microsoft said Friday that it is halting devel...,Microsoft will stop developing versions of its...,0.684402,0


## Part 2 - Topic Modelling

For the second part of this assignment, you will use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. You will first need to finish the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. Extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [25]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [72]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

# Your code here:
ldamodel =  gensim.models.ldamodel.LdaModel (corpus, num_topics = 10, id2word = id_map, passes =25, random_state =34)

### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.014*"sci"')`

for example.

*This function should return a list of tuples.*

In [80]:
def lda_topics():

    result = ldamodel.print_topics(10,10)
    
    return result

### topic_distribution

For the new document `new_doc`, find the topic distribution. Remember to use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function should return a list of tuples, where each tuple is `(#topic, probability)`*

In [59]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [75]:
def topic_distribution():
    
    X_new = vect.transform(new_doc)

    # Convert sparse matrix to gensim corpus.
    corpus_new = gensim.matutils.Sparse2Corpus(X_new, documents_columns=False)

    a =ldamodel.get_document_topics(corpus_new)
    list1 = [w for w in a]
    
    return list1 # Your Answer Here

In [82]:
topic_distribution()

[[(0, 0.02000183),
  (1, 0.020002052),
  (2, 0.020000003),
  (3, 0.49635696),
  (4, 0.02000277),
  (5, 0.020002862),
  (6, 0.0200017),
  (7, 0.02000137),
  (8, 0.020001847),
  (9, 0.34362862)]]

### topic_names

From the list of the following given topics, assign topic names to the topics you found. If none of these names best matches the topics you found, create a new 1-3 word "title" for the topic.

Topics: Health, Science, Automobiles, Politics, Government, Travel, Computers & IT, Sports, Business, Society & Lifestyle, Religion, Education.

*This function should return a list of 10 strings.*

In [83]:
def topic_names():
    
    # Your Code Here
    
    return ['Education', 'Science', 'Computers & IT', 'Religion', 'Automobiles', 'Sports', 'Health', 'Society & Lifestyle', 'Business', 'Space']

In [85]:
lda_topics()

[(0,
  '0.056*"edu" + 0.043*"com" + 0.033*"thanks" + 0.022*"mail" + 0.021*"know" + 0.020*"does" + 0.014*"info" + 0.012*"monitor" + 0.010*"looking" + 0.010*"don"'),
 (1,
  '0.024*"ground" + 0.018*"current" + 0.018*"just" + 0.013*"want" + 0.013*"use" + 0.011*"using" + 0.011*"used" + 0.010*"power" + 0.010*"speed" + 0.010*"output"'),
 (2,
  '0.061*"drive" + 0.042*"disk" + 0.033*"scsi" + 0.030*"drives" + 0.028*"hard" + 0.028*"controller" + 0.027*"card" + 0.020*"rom" + 0.018*"floppy" + 0.017*"bus"'),
 (3,
  '0.023*"time" + 0.015*"atheism" + 0.014*"list" + 0.013*"left" + 0.012*"alt" + 0.012*"faq" + 0.012*"probably" + 0.011*"know" + 0.011*"send" + 0.010*"months"'),
 (4,
  '0.025*"car" + 0.016*"just" + 0.014*"don" + 0.014*"bike" + 0.012*"good" + 0.011*"new" + 0.011*"think" + 0.010*"year" + 0.010*"cars" + 0.010*"time"'),
 (5,
  '0.030*"game" + 0.027*"team" + 0.023*"year" + 0.017*"games" + 0.016*"play" + 0.012*"season" + 0.012*"players" + 0.012*"win" + 0.011*"hockey" + 0.011*"good"'),
 (6,
  '0.0